### Detecting Deep Fakes using Neural Networks

Importing data

In [ ]:
!wget -nv -q --show-progress -O ff.zip https://bit.ly/3w0xyBl
!unzip -q ff.zip -d ffdata
!rm ff.zip

ff.zip              100%[===================>]   1.75G  37.4MB/s    in 53s     


Importing libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

from tensorflow import keras

Preparing image data into TF datasets

In [ ]:
train_folder = '/content/ffdata/train'
val_folder = '/content/ffdata/val'

df_train = pd.read_csv(train_folder + '/image_labels.csv')
df_val = pd.read_csv(val_folder + '/image_labels.csv')


datagen = ImageDataGenerator(rescale=1./255)
train_set = datagen.flow_from_dataframe(dataframe=df_train, directory=train_folder, classes= ['real', 'fake'], class_mode="binary", target_size=(128, 128), batch_size=32)
val_set = datagen.flow_from_dataframe(dataframe=df_val, directory=val_folder, classes= ['real', 'fake'], class_mode="binary", target_size=(128, 128), batch_size=32)

print("Check class name mapping to label index:")
print(train_set.class_indices)
print(val_set.class_indices)

Found 66722 validated image filenames belonging to 2 classes.
Found 12592 validated image filenames belonging to 2 classes.
Check class name mapping to label index:
{'fake': 0, 'real': 1}
{'fake': 0, 'real': 1}


Defining callbacks

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
mc = ModelCheckpoint("/content/ffdata/checkpoint", monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

Defining and training the model

In [ ]:
model = keras.Sequential([
   keras.layers.Conv2D(8, 3, padding='same', activation='relu', input_shape=(128,128,3)),
   keras.layers.BatchNormalization(),
   #
   keras.layers.Conv2D(8, 5, padding='same', activation='relu'),
   keras.layers.BatchNormalization(),
   keras.layers.MaxPool2D(2,2),
   #
   keras.layers.Conv2D(16, 5, padding='same', activation='relu'),
   keras.layers.BatchNormalization(),
   keras.layers.MaxPool2D(2,2),
   #
   keras.layers.Conv2D(16, 5, padding='same', activation='relu'),
   keras.layers.BatchNormalization(),
   keras.layers.MaxPool2D(4,4),
   #
   keras.layers.Flatten(),
   keras.layers.Dropout(0.5),
   keras.layers.Dense(16),
   keras.layers.Dropout(0.5),
   keras.layers.Dense(1, activation='sigmoid')
])

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=10**-3,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
model.summary()
model.fit(train_set, class_weight={0: 1., 1:5.}, steps_per_epoch= None, epochs=50, validation_data=val_set, validation_steps=10, callbacks=[mc,es])

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_64 (Conv2D)           (None, 128, 128, 8)       224       
_________________________________________________________________
batch_normalization_64 (Batc (None, 128, 128, 8)       32        
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 128, 128, 8)       1608      
_________________________________________________________________
batch_normalization_65 (Batc (None, 128, 128, 8)       32        
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 64, 64, 16)        3216      
_________________________________________________________________
batch_normalization_66 (Batc (None, 64, 64, 16)      

Using a helper function to generate model predictions on test data and store the predictions in a CSV format

In [ ]:
def read_image_from_disk(path):

  """
  Helper function to read image from disk given a absolute path.

  :param path: Absolute path to image file on disk
  :return: Image in Numpy Ndarray representation
  """

  img = tf.keras.preprocessing.image.load_img(path, target_size=(128,128,3))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = img/255
  img = np.expand_dims(img, axis=0)

  return img


def get_frames_to_vid_mapping(frame_list):

  """
  Helper function to generate a mapping of frames to it's corresponding video 
  name.

  The path of frames in the frame_list will be in such format:
  image/[video name]/[frame number].jpg
  e.g. image/00000/00032.jpg

  :param frame_list: A list of paths to the image frames
  :return: A sorted dictionary with keys as the video name and value as the
           corresponding frames.
           e.g. of returned mapping dictionary:

            {
              "00000":[
                  "00032",
                  "00064",
                  .
                  .
                  .
                  "00487"
              ],
              "00001":[
                  "00000",
                  "00032",
                  .
                  .
                  .
                  "00392"
              ],
              .
              .
              .
              "00790":[
                  "00000",
                  "00027",
                  .
                  .
                  .
                  "00542"
              ]
            }
  """

  # Get all videos name
  vidnames = [frame.split("/")[1:2][0] for frame in frame_list]
  # Get only unique names
  vidnames = set(vidnames)
  # Init the mapping dict
  mapping = {vidname: [] for vidname in vidnames}

  # Add frames to to its corresponding list
  for frame in frame_list:
    vidname = str(frame.split("/")[1:2][0])
    frame_number = str(frame.split("/")[-1].split(".")[0])
    mapping[vidname].append(frame_number)

  return dict(sorted(mapping.items()))


def infer_videos(test_data_path, csv_file, num_of_videos='All'):

  """
  Function to infer a test data set. The function takes in a path to the test
  data set and a csv file that contain the paths of the frames extracted from 
  the videos in the test dataset.

  :param test_data_path: Absoulute path to the test dataset
  :param csv_file: File Name of the CSV file that must be in the test_data_path
  :param num_of_videos: Number of videos to infer from the dataset (default: All)
  :return: Pandas dataframe which contains the prediction (probability of being 
           fake) of each video. 
  """

  list_dir = list(pd.read_csv(test_data_path + csv_file).iloc[:,0])

  mapping = get_frames_to_vid_mapping(list_dir)

  # [*mapping] gives the list of keys (video name) in the mapping dict
  num_of_videos_avail = len([*mapping])

  # Set number of videos to be inferred to total of videos available if given 
  # num_of_videos is more than max amount of available videos
  if num_of_videos == 'All' or num_of_videos > num_of_videos_avail:
      num_of_videos = num_of_videos_avail

  # init mapping of videos to its corresponding predicted probabilities
  videos_to_prediction = {}

  # Loop through each video and make a prediction of each frame in the video.
  # Assigned a prediction to each video by taking the mean of its corresponding
  # frames' probabilities.
  for video_name in [*mapping][0:num_of_videos]:

    frames = mapping[video_name]
    predictions = []
    print("Infering video {video}...".format(video=video_name))
    print("Processing frame ", end=" ")

    # Process each frame in video
    for frame in frames:
      print(frame, end =", ")
      frame_path = "image/{video_name}/{frame}.jpg".format(video_name=video_name, frame=frame)
      img = read_image_from_disk(test_data_path + frame_path)
      prediction = model(img)[0]
      # Collect only the 'real' side of probability
      predictions.append(prediction[0])

    # Take the mean of the probabilities from the frames
    videos_to_prediction[video_name] = np.median(predictions)
    print("Done!")
  
  return pd.DataFrame(videos_to_prediction.items())

In [ ]:
modelPredictions = infer_videos("/content/ffdata/test/", "image_labels.csv")
print(modelPredictions)

Infering video 00000...
Processing frame  00032, 00064, 00097, 00129, 00162, 00194, 00227, 00259, 00292, 00324, 00357, 00389, 00422, 00454, 00487, Done!
Infering video 00001...
Processing frame  00000, 00032, 00065, 00098, 00130, 00163, 00196, 00228, 00261, 00294, 00326, 00359, 00392, Done!
Infering video 00002...
Processing frame  00000, 00027, 00054, 00081, 00108, 00135, 00162, 00189, 00216, 00243, 00271, 00298, 00325, 00352, 00379, 00406, 00433, 00460, 00487, 00514, 00542, Done!
Infering video 00003...
Processing frame  00000, 00031, 00063, 00095, 00127, 00159, 00191, 00222, 00254, 00286, 00318, 00350, 00382, 00413, 00445, 00477, 00509, 00541, 00573, 00605, Done!
Infering video 00004...
Processing frame  00000, 00026, 00052, 00078, 00105, 00131, 00157, 00183, 00210, 00236, 00262, 00289, 00315, 00341, 00367, 00394, 00420, 00446, 00472, 00499, 00525, 00551, 00578, 00604, 00630, 00656, 00683, 00709, 00735, 00762, Done!
Infering video 00005...
Processing frame  00000, 00025, 00051, 0007

Downloading the CSV

In [ ]:
modelPredictions.columns = ['vid_name', 'label']
modelPredictions.to_csv("/content/ffdata/Lee Pak Shuang_pakshuang@gmail.com.csv", index=False)